# Waste identification with instance segmentation in TensorFlow

Welcome to the Instance Segmentation Colab! This notebook will take you through the steps of running an "out-of-the-box" Mask RCNN Instance Segmentation model on images.

To finish this task, a proper path for the saved models and a single image needs to be provided. The path to the labels on which the models are trained is in the waste_identification_ml directory inside the Tensorflow Model Garden repository. The label files are inferred automatically for both models.

## Imports and Setup

In [ ]:
from six.moves.urllib.request import urlopen
from six import BytesIO
from PIL import Image
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt
import matplotlib
import logging
import pandas as pd
from labels import load_labels

logging.disable(logging.WARNING)

Run the following cell to import utility functions that will be needed for pre-processing, post-processing and color detection.



To visualize the images with the proper detected boxes and segmentation masks, we will use the TensorFlow Object Detection API. To install it we will clone the repo.



In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3985, done.
remote: Counting objects: 100% (3985/3985), done.
remote: Compressing objects: 100% (3093/3093), done.
remote: Total 3985 (delta 1151), reused 1942 (delta 835), pack-reused 0
Receiving objects: 100% (3985/3985), 49.76 MiB | 33.28 MiB/s, done.
Resolving deltas: 100% (1151/1151), done.


In [ ]:
sys.path.append('models/research/')
from object_detection.utils import visualization_utils as viz_utils
%matplotlib inline

## Utilities

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (1, h, w, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


def load_model(model_handle):
    """Loads a TensorFlow SavedModel and returns a function that can be used to make predictions.

    Args:
      model_handle: A path to a TensorFlow SavedModel.

    Returns:
      A function that can be used to make predictions.
    """
    print('loading model...')
    print(model_handle)
    model = tf.saved_model.load(model_handle)
    print('model loaded!')
    detection_fn = model.signatures['serving_default']
    return detection_fn


def perform_detection(model, image):
  """Performs Mask RCNN on an image using the specified model.

  Args:
    model: A function that can be used to make predictions.
    image_np: A NumPy array representing the image to be detected.

  Returns:
    A list of detections.
  """
  detection_fn = model(image)
  detection_fn = {key: value.numpy() for key, value in detection_fn.items()}
  return detection_fn

In [ ]:
# 'material_model' output is both material and its sub type e.g. Plastics_PET
# 'material_form_model' outputs the form of an object e.g. can, bottle, etc
MODEL_WEIGHTS = {
'material_url' : 'https://storage.googleapis.com/tf_model_garden/vision/waste_identification_ml/two_model_strategy/material/material_version_1.zip',
'material_form_url' : 'https://storage.googleapis.com/tf_model_garden/vision/waste_identification_ml/two_model_strategy/material_form/material_form_version_1.zip',
}

ALL_MODELS = {
'material_model' : 'material/two_model_material_1_saved/saved_model/',
'material_form_model' : 'material_form/two_model_material_form_1_saved/saved_model/',
}

LABELS = {
'material_model' : 'models/official/projects/waste_identification_ml/pre_processing/config/data/two_model_strategy_material.csv',
'material_form_model' : 'models/official/projects/waste_identification_ml/pre_processing/config/data/two_model_strategy_material_form.csv',
}

# path to a sample image stored in the repo
IMAGES_FOR_TEST = {
  'Image1' : 'models/official/projects/waste_identification_ml/pre_processing/config/sample_images/image_2.png'
}

## Import pre-trained models.

In [ ]:
# download the model weights from the Google's repo
url1 = MODEL_WEIGHTS['material_url']
url2 = MODEL_WEIGHTS['material_form_url']
!python download_and_unzip_models.py --material_url $url1 material_form_url $url2

## Load label map data

Label maps correspond index numbers to category names, so that when our convolution network predicts 5, we know that this corresponds to airplane. Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

We are going, for simplicity, to load from the repository that we loaded the Object Detection API code

In [ ]:
# the total number of predicted labels (category_indices) for a combined model = 741
category_indices, category_index = load_labels(LABELS)

In [ ]:
# display labels only for 'material' model
# total number of labels for 'material' model = 19
category_indices[0]

['Na',
 'Fiber_Na',
 'Food_Na',
 'Glass_Na',
 'Inorganic-wastes_Na',
 'Metals_Na',
 'Plastics_HDPE',
 'Plastics_LDPE',
 'Plastics_Others-HIPC',
 'Plastics_Others-MLP',
 'Plastics_Others-Tetrapak',
 'Plastics_PET',
 'Plastics_PP',
 'Plastics_PS',
 'Plastics_PVC',
 'Rubber-&-Leather_Na',
 'Textiles_Na',
 'Wood_Na',
 'Yard-trimming_Na']

In [ ]:
# display labels only for 'material_form' model
# total number of labels for 'material form' model = 39
category_indices[1]

['Na',
 'Bag',
 'Battery',
 'Blister-pack',
 'Book-&-magazine',
 'Bottle',
 'Box',
 'Brush',
 'Bulb',
 'Can',
 'Cards',
 'Carton',
 'Cassette-&-tape',
 'Clamshell',
 'Clothes',
 'Container',
 'Cosmetic',
 'Cup-&-glass',
 'Cutlery',
 'Electronic-devices',
 'Flexibles',
 'Foil',
 'Foot-wear',
 'Hangers',
 'Jug-&-Jar',
 'Lid',
 'Mirror',
 'Office-Stationary',
 'Paper-Products-Others',
 'Paper-Products-Others-Cardboard',
 'Paper-Products-Others-Newspaper',
 'Paper-Products-Others-Whitepaper',
 'Pipe',
 'Sachets-&-Pouch',
 'Scissor',
 'Tangler',
 'Toys',
 'Tray',
 'Tube']

## Load pre-trained weights for both models.

In [ ]:
# loading both models
detection_fns = [load_model(model_path) for model_path in ALL_MODELS.values()]

loading model...
material/two_model_material_1_saved/saved_model/
model loaded!
loading model...
material_form/two_model_material_form_1_saved/saved_model/
model loaded!
